In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Change depending where the evaluation csv results are per person and llm
directory = '/content/drive/My Drive/LLM-Data/evaluate-test/GPT4_Enslaved_MainAgent'


In [ ]:
import os
import pandas as pd

In [ ]:
def is_good_result(row, thresholds):
    print(row[['Predicate FuzzyWuzzy Ratio', 'Predicate FuzzyWuzzy Token Set Ratio',
               'Predicate Jaro-Winkler Similarity', 'Predicate Cosine Similarity',
               'Object FuzzyWuzzy Ratio', 'Object FuzzyWuzzy Token Set Ratio',
               'Object Jaro-Winkler Similarity', 'Object Cosine Similarity']])
    return (
        row['Predicate FuzzyWuzzy Ratio'] >= thresholds['Predicate FuzzyWuzzy Ratio'] or
        row['Predicate FuzzyWuzzy Token Set Ratio'] >= thresholds['Predicate FuzzyWuzzy Token Set Ratio'] or
        row['Predicate Jaro-Winkler Similarity'] >= thresholds['Predicate Jaro-Winkler Similarity'] or
        row['Predicate Cosine Similarity'] >= thresholds['Predicate Cosine Similarity'] or
        row['Object FuzzyWuzzy Ratio'] >= thresholds['Object FuzzyWuzzy Ratio'] or
        row['Object FuzzyWuzzy Token Set Ratio'] >= thresholds['Object FuzzyWuzzy Token Set Ratio'] or
        row['Object Jaro-Winkler Similarity'] >= thresholds['Object Jaro-Winkler Similarity'] or
        row['Object Cosine Similarity'] >= thresholds['Object Cosine Similarity']
    )
# If at least one of the metrics if above the threshold


In [ ]:
def process_csv(file_path, thresholds):

    df = pd.read_csv(file_path, delimiter=',')

    # Print column names for debugging
    print("Columns in CSV:", df.columns.tolist())

    df.columns = df.columns.str.strip()

    # Get unique triples based on TSV Predicate and TXT Predicate
    unique_triples = df[['Truth Predicate', 'Truth Object', 'Generated Predicate']].drop_duplicates()

    # Total number of unique comparisons (TSV Predicate compared with TXT Predicate)
    total_unique_triples = len(unique_triples)

    # Join unique_triples back with original dataframe to keep metric columns for evaluation
    merged_df = pd.merge(unique_triples, df, how='left', on=['Truth Predicate', 'Truth Object', 'Generated Predicate'])

    # Count the number of "good" comparisons based on the thresholds
    good_triples = merged_df.apply(lambda row: is_good_result(row, thresholds), axis=1).sum()

    # Calculate percentage of good comparisons
    percentage_good = (good_triples / total_unique_triples) * 100 if total_unique_triples > 0 else 0

    return total_unique_triples, good_triples, percentage_good


In [ ]:
def process_directory(directory, thresholds, output_file):
    with open(output_file, 'w') as f:
        for filename in os.listdir(directory):
            if filename.endswith('.csv'):
                file_path = os.path.join(directory, filename)
                total_unique_triples, good_triples, percentage_good = process_csv(file_path, thresholds)


                f.write(f'File: {filename}\n')
                f.write(f'Total Unique Comparisons (TSV1 vs TSV2 Predicates): {total_unique_triples}\n')
                f.write(f'Good Comparisons: {good_triples}\n')
                f.write(f'Percentage Good: {percentage_good:.2f}%\n\n')


In [ ]:
thresholds = {
    'Predicate FuzzyWuzzy Ratio': 80,
    'Predicate FuzzyWuzzy Token Set Ratio': 80,
    'Predicate Jaro-Winkler Similarity': 0.8,
    'Predicate Cosine Similarity': 0.8,
    'Object FuzzyWuzzy Ratio': 80,
    'Object FuzzyWuzzy Token Set Ratio': 80,
    'Object Jaro-Winkler Similarity': 0.8,
    'Object Cosine Similarity': 0.8
}


In [ ]:
output_file = '/content/drive/My Drive/LLM-Data/percentages'

# Process the directory and write results to the output file
process_directory(directory, thresholds, output_file)

IsADirectoryError: [Errno 21] Is a directory: '/content/drive/My Drive/LLM-Data/percentages-truth-vs-generated'

In [ ]:
import os
import pandas as pd

def is_good_result(row, thresholds):
    """Check if any similarity metric exceeds the specified thresholds."""
    print(row[['Predicate Fuzzy Ratio',
                'Predicate Jaro-Winkler',
                'Predicate Cosine Similarity',
                'Object Fuzzy Ratio',
                'Object Jaro-Winkler',
                'Object Cosine Similarity']])
    return (
        row['Predicate Fuzzy Ratio'] >= thresholds['Predicate Fuzzy Ratio'] or
        row['Predicate Jaro-Winkler'] >= thresholds['Predicate Jaro-Winkler'] or
        row['Predicate Cosine Similarity'] >= thresholds['Predicate Cosine Similarity'] or
        row['Object Fuzzy Ratio'] >= thresholds['Object Fuzzy Ratio'] or
        row['Object Jaro-Winkler'] >= thresholds['Object Jaro-Winkler'] or
        row['Object Cosine Similarity'] >= thresholds['Object Cosine Similarity']
    )

def process_csv(file_path, thresholds):
    """Process the CSV file to calculate good comparisons."""
    if os.stat(file_path).st_size == 0:
        print(f"Skipping empty file: {file_path}")
        return 0, 0, 0  # Return zeros if the file is empty

    df = pd.read_csv(file_path)

    # Print column names for debugging
    print("Columns in CSV:", df.columns.tolist())

    # Strip whitespace from column names
    df.columns = df.columns.str.strip()

    # Get unique triples based on Truth Predicate and Truth Object
    unique_triples = df[['Truth Predicate', 'Truth Object']].drop_duplicates()

    # Total number of unique comparisons
    total_unique_triples = len(unique_triples)

    # Count unique triples with good matches
    good_matches = df[df.apply(lambda row: is_good_result(row, thresholds), axis=1)]
    unique_good_matches = good_matches[['Truth Predicate', 'Truth Object']].drop_duplicates()
    num_good_matches = len(unique_good_matches)

    # Calculate percentage of good comparisons based on good matches
    percentage_good = (num_good_matches / total_unique_triples) * 100 if total_unique_triples > 0 else 0

    return total_unique_triples, num_good_matches, percentage_good

def process_directory(directory, thresholds, output_directory):
    """Process all CSV files in the specified directory and write results to individual output files."""
    os.makedirs(output_directory, exist_ok=True)  # Create output directory if it doesn't exist

    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            file_path = os.path.join(directory, filename)
            total_unique_triples, good_triples, percentage_good = process_csv(file_path, thresholds)

            # Create output file name
            output_file_name = f'evaluation-{os.path.splitext(filename)[0]}.txt'  # Prefix with "evaluation-"
            output_file_path = os.path.join(output_directory, output_file_name)


            if total_unique_triples > 0:
                with open(output_file_path, 'w') as f:
                    f.write(f'File: {filename}\n')
                    f.write(f'Total Unique Comparisons (Truth Predicate vs Truth Object): {total_unique_triples}\n')
                    f.write(f'Good Matches: {good_triples}\n')
                    f.write(f'Percentage of Good Matches: {percentage_good:.2f}%\n\n')

# Thresholds
thresholds = {
    'Predicate Fuzzy Ratio': 80,
    'Predicate Jaro-Winkler': 0.8,
    'Predicate Cosine Similarity': 0.8,
    'Object Fuzzy Ratio': 80,
    'Object Jaro-Winkler': 0.8,
    'Object Cosine Similarity': 0.8
}

# Directory containing the CSV files and the output directory, change depending where the data is
directory = '/content/drive/My Drive/LLM-Data/evaluate-test/GPT4_Enslaved_MainAgent'
output_directory = '/content/drive/My Drive/LLM-Data/percentages/GPT4_Enslaved_MainAgent'


process_directory(directory, thresholds, output_directory)


Streaming output truncated to the last 5000 lines.
Name: 190, dtype: object
Predicate Fuzzy Ratio                25
Predicate Jaro-Winkler         0.481481
Predicate Cosine Similarity         0.0
Object Fuzzy Ratio                   18
Object Jaro-Winkler            0.473016
Object Cosine Similarity            0.0
Name: 191, dtype: object
Predicate Fuzzy Ratio                36
Predicate Jaro-Winkler         0.607143
Predicate Cosine Similarity         0.0
Object Fuzzy Ratio                   20
Object Jaro-Winkler            0.488889
Object Cosine Similarity            0.0
Name: 192, dtype: object
Predicate Fuzzy Ratio                36
Predicate Jaro-Winkler         0.607143
Predicate Cosine Similarity         0.0
Object Fuzzy Ratio                   31
Object Jaro-Winkler            0.558153
Object Cosine Similarity            0.0
Name: 193, dtype: object
Predicate Fuzzy Ratio                36
Predicate Jaro-Winkler         0.607143
Predicate Cosine Similarity         0.0
Object Fu

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/LLM-Data/evaluate-test'